In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/kaggle/input/shopee-code-league-20/_DA_Order_Brushing/order_brush_order.csv',parse_dates=['event_time'])
data[['orderid','shopid','userid']] = data[['orderid','shopid','userid']].astype(str)

In [ ]:
display(data.dtypes)
data

In [ ]:
#因為可能有同商店同時間不同訂單發生的可能，所以用這樣的方式做groupby，順便整合資料
agg_func = [lambda ser:ser.to_list(),'count']
a = data.groupby(['shopid','event_time']).agg({'userid':agg_func,'orderid':agg_func})
a.columns = ['_'.join(x).replace('<lambda_0>','list') for x in a.columns]

In [ ]:
a

In [ ]:
#注意會有同商店同時間不同訂單的情況
print(a.query('userid_count != 1').shape[0])
#這邊可以知道沒有不同buyer共用同一個orderid的bug
print(a.query('userid_count != orderid_count').shape[0])

In [ ]:
#檢查userid的總數有沒有缺少，如果和原來的不一樣就不對了
assert a['userid_count'].sum() == data.shape[0]
print('pass')

In [ ]:
a = a.reset_index()

In [ ]:
answer = {}
#固定一個商店
for i,shopid in enumerate(a['shopid'].unique()):
    print(i,'\t',shopid)
    order_brushing_period = pd.DataFrame(columns=['userid_list'])
    shop_df = a.query(f'shopid == "{shopid}"')[['event_time','userid_list']]
    shop_df = shop_df.set_index('event_time')
    shop_df = shop_df.sort_index()
    #固定某個時間點
    for event_time in shop_df.index:
        #選取距離該時間點一小時以內的資料
        current = shop_df.loc[event_time:].first(pd.Timedelta('1H'))#包含起始時間但不包含截止時間
        #因為有可能同一時間內有多筆資料，而之後也有這個時間點內的buyer這樣如果是單純把userid_count給sum起來有可能會重複算，
        #所以最安全只能從userid_list一個一個合併起來算
        users = []
        for userid_list in current['userid_list']:
            users.extend(userid_list)
        #統計這個時間裡面每個相異的buyer出現幾次
        users = pd.Series(users,dtype='object').value_counts()
        #求得這段時間內實際上有多少個不同的buyer
        unique_users_count = users.shape[0]
        #計算concentrate_rate=這段時間的交易筆數/多少個不同的buyer
        concentrate_rate = current.shape[0]/unique_users_count
        #如果concentrate_rate>=3判定該時段有order_brushing_period，要記錄起來
        #我猜重疊的order_brushing_period不會重複計算，所以用combine_first來登記，如果要重覆計算要用append
        if concentrate_rate >= 3:
            #登記這個時段到order_brushing_period裡面
            order_brushing_period = order_brushing_period.combine_first(current)
    #找出所有order_brushing_period裡訂單最多的人，如果裡面有多個人的訂單是一樣多的，而且也都是最多的，那就要都算進去
    #所以找出訂單數量最大值，再找出有哪些人的訂單這麼多，全部登記起來
    suspicious = []
    for userid_list in order_brushing_period['userid_list']:
        suspicious.extend(userid_list)
    #統計所有order_brushing_period裡，不同的userid有多少訂單
    suspicious = pd.Series(suspicious,dtype='object').value_counts()
    #找出最多的訂單數量
    max_count = users.max()
    #找出所有相同數量的userid
    suspicious = suspicious.loc[suspicious==max_count]
    #由小排到大
    suspicious = suspicious.sort_index()
    #依照比賽格式合併成字串
    suspicious = '&'.join(suspicious.index)
    #依照比賽說如果沒有的話填0
    if suspicious == '':
        suspicious = '0'
    answer[shopid] = suspicious

In [ ]:
answer = pd.Series(answer)

In [ ]:
answer[answer!='0']